### 예스24 크롤링

In [ ]:
# webdriver-manager 라이브러리 불러오기

from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

In [ ]:
from selenium import webdriver

browser = webdriver.Chrome()

In [ ]:
from selenium.webdriver.common.by import By
url = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)

In [ ]:
# 1 페이지의 링크 데이터 전부 수집

### 한 개의 베스트 셀러 링크 수집
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')  # 요소

### 한 페이지 전체의 링크 데이터
datas = browser.find_elements(By.CLASS_NAME, 'gd_name')  # 리스트

for i in datas :
    print(i.get_attribute('href'))


In [ ]:
# 3 페이지까지의 링크 데이터 전부 수집
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24    1페이지
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=2&pageSize=24    2페이지
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24    3페이지

import time

link_list = []
for i in range(1, 4) :
    print('*' * 10, f'{i}페이지 수집 중입니다.', '*' * 10)
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)

    # 링크데이터 수집
    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

    ### 한 페이지 전체의 링크 데이터
    datas = browser.find_elements(By.CLASS_NAME, 'gd_name')  # 리스트

    for i in datas :
        link = i.get_attribute('href')   # print문 해놨다가 에러 발생

        link_list.append(link)

    time.sleep(3)

    # print(link_list)

In [ ]:
len(link_list)

In [ ]:
# 데이터 베이스 연동 후 -> 수집한 데이터를  DB에 저장 (csv)

import pymysql
import time
import re
from datetime import datetime

# 연결
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='QOR0625dps@em02!',
    db='yes24',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

# 접속
with conn.cursor() as cur :
    for link in link_list :     
        browser.get(link)
        
        # title(제목)
        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        # author(저자)
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        # publisher(출판사)
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
        # publishing(출판일)
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text  # 데이터 형식
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        if match :
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime('%Y-%m-%d') # Y 연도만 대문자 사용
        else :
            publishing = '2023-01-01'

        # rating(평점)
        rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
        if rating == '' :
            rating = 0
        # review(리뷰)
        review = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').text
        if len(review) == 15 :
            review = 0
        else :
            review = review.split('(')[1][:-2]
            review = int(review.replace(',', ''))
        # sales(판매지수)
        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(' ')[2]
        sales = int(sales.replace(',', ''))
        # price(가격)
        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(',', ''))
        # ranking(국내도서랭킹)
        # ranking_weeks(국내도서TOP100)
        full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
        parts = full_text.split(' | ')

        if len(parts) == 1:
            ranking = 0
            ranking_weeks = 0
        else:
            try:
                ranking_part = parts[0]
                ranking = ''.join(filter(str.isdigit, ranking_part))
            except:
                ranking = 0

            try:
                # 국내도서 top20 3주
                ranking_weeks_part = parts[1]
                ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))
            except:
                ranking_weeks = 0

        # sql문 작성    
        sql = '''INSERT INTO Books(title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks)
                VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
        # 수행
        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        # 반영
        conn.commit()

        time.sleep(2)